In [27]:
# Import
import requests
import json
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
#Set credentials 
baseURL = "https://newsapi.org/v2/everything?"
total_requests=2
verbose=True

# THIS CODE WILL NOT WORK UNLESS YOU INSERT YOUR API KEY IN THE NEXT LINE
API_KEY='9cf6d393469e41738d84303a9c840fd1'


In [29]:
TOPICS = ['Cardiovascular', 'Cancer','Stroke','Diabetes']

In [30]:
def extract (x):

    URLpost = {'apiKey': API_KEY,
                'q': '+'+ TOPICS[x],
                'sortBy': 'relevancy',
                'totalRequests': 1}

    # print(baseURL)
    # print(URLpost)

    #GET DATA FROM API
    response = requests.get(baseURL, URLpost) #request data from the server
    # print(response.url);  
    response = response.json() #extract txt data from request into json

    # PRETTY PRINT
    # https://www.digitalocean.com/community/tutorials/python-pretty-print-json

    # print(json.dumps(response, indent=2))

    # #GET TIMESTAMP FOR PULL REQUEST
    from datetime import datetime
    timestamp = datetime.now().strftime("%Y-%m-%d-H%H-M%M-S%S")

    # SAVE TO FILE 
    with open(timestamp+'-newapi-raw-data.json', 'w') as outfile:
        json.dump(response, outfile, indent=4)
        
    return response



In [31]:
response1 = extract(0)
response2 = extract(1)
response3 = extract(2)
response4 = extract(3)

In [32]:
# Utility function
# Function to clean strings
def string_cleaner(input_string):
    try: 
        out=re.sub(r"""
                    [,.;@#?!&$-]+  # Accept one or more copies of punctuation
                    \ *           # plus zero or more copies of a space,
                    """,
                    " ",          # and replace it with a single space
                    input_string, flags=re.VERBOSE)

        #REPLACE SELECT CHARACTERS WITH NOTHING
        out = re.sub('[’.]+', '', input_string)

        #ELIMINATE DUPLICATE WHITESPACES USING WILDCARDS
        out = re.sub(r'\s+', ' ', out)

        #CONVERT TO LOWER CASE
        out=out.lower()
    except:
        print("ERROR")
        out=''
    return out

In [33]:
def clean (response):
    article_list=response['articles']   #list of dictionaries for each article
    article_keys=article_list[0].keys()
    # print("AVAILABLE KEYS:")
    # print(article_keys)
    index=0
    cleaned_data=[];  
    for article in article_list:
        tmp=[]
        # if(verbose):
        #     print("#------------------------------------------")
        #     print("#",index)
        #     print("#------------------------------------------")

        for key in article_keys:
            # if(verbose):
            #     print("----------------")
            #     print(key)
            #     print(article[key])
            #     print("----------------")

            # if(key=='source'):
            #     src=string_cleaner(article[key]['name'])
            #     tmp.append(src) 

            # if(key=='author'):
            #     author=string_cleaner(article[key])
            #     #ERROR CHECK (SOMETIMES AUTHOR IS SAME AS PUBLICATION)
            #     if(src in author): 
            #         print(" AUTHOR ERROR:",author);author='NA'
            #     tmp.append(author)

            # if(key=='title'):
            #     tmp.append(string_cleaner(article[key]))
                
            if(key=='description'):
                tmp.append(string_cleaner(article[key]))

            # if(key=='content'):
            #     tmp.append(string_cleaner(article[key]))

            # if(key=='publishedAt'):
            #     #DEFINE DATA PATERN FOR RE TO CHECK  .* --> wildcard
            #     ref = re.compile('.*-.*-.*T.*:.*:.*Z')
            #     date=article[key]
            #     if(not ref.match(date)):
            #         print(" DATE ERROR:",date); date="NA"
            #     tmp.append(date)

        cleaned_data.append(tmp)
        index+=1

    return cleaned_data

In [34]:
# Create DataFrame
cleaned_data1 = clean(response1)
df1 = pd.DataFrame(cleaned_data1)
cleaned_data2 = clean(response2)
df2 = pd.DataFrame(cleaned_data2)
cleaned_data3 = clean(response3)
df3 = pd.DataFrame(cleaned_data3)
cleaned_data4 = clean(response4)
df4 = pd.DataFrame(cleaned_data4)
df1['label'] = 1
df2['label'] = 0
df3['label'] = 0
df4['label'] = 0



In [35]:
cardiovascular_text = pd.concat([df1, df2, df3, df4])
cardiovascular_text = cardiovascular_text.rename(columns={0: 'text'})
cardiovascular_text.head()


,text,label
0,researchers increasingly find that the effects...,1
1,"together, ai imaging and ai genetic analysis m...",1
2,las enfermedades relacionadas con nuestro sist...,1
3,a monthly dose of the vitamin was found to low...,1
4,"author cat bohannon says there's a ""male norm""...",1


In [36]:
cardiovascular_text.shape

(400, 2)

In [37]:
cardiovascular_text.to_csv('../data/01-modified-data/cardiovascular_text_final.csv')